# Bildklassifikation mit dem MURA-Datensatz

Bei der Klassifikation geht es darum, einem Bild ein Label aus einer vorgegebenen Menge an möglichen Labels zuzuweisen. Im Vergleich zur Segmentierung und Objekt Detection ist diese Aufgabe die einfachste.

In diesem Notebook wird ein Klassifikationsmodell für den MURA-Datensatz mit Keras und Tensorflow als Backend trainiert.

## Datensatz

Der [MURA](https://stanfordmlgroup.github.io/competitions/mura/) (**mu**sculoskeletal **ra**diographs) Datensatz besteht aus 40.561 Röntgenbildern von Oberkörperextremitäten der Kategorien: Ellbogen, Finger, Unterarm, Hand, Oberarm, Schulter, und Handgelenk. Zusätzlich ist jedem Bild das Label normal oder abnormal zugeordnet.

![XR_HAND](http://s1.saviola.de/screens/mura.png)

In diesem Jupyter-Notebook soll ein Model trainiert werden welches Bilder des MURA-Datensatzes in eine der sieben Extremitätsklassen einordnen kann.

## Jupyter-Notebook

Jupyter-Notebooks sind interaktive Python-Scripte, in denen Markdown und sogar Latex zur Dokumentation verwendet werden kann. In diesem Abschnitt sollen einerseits allgemeine Informationen wie Tastenkombinationen und übliche Workflows, andererseits aber auch für diesen Workshop und die verwendete Hardware spezifische Informationen im Umgang mit Python und Jupyter-Notebooks geliefert werden.

### Wichtiges auf einen Blick (a.k.a. TL;DR)

- **beim Wechseln auf ein anderes Notebook immer den Kernel beenden (Kernel -> Shutdown)**
- **Shift + Enter** zum Ausführen der aktiven Zelle
- **Strg + Shift + p** zum Öffnen der Kommandopalette
- **Shift + o** zum Togglen des Zell-Scrollings
- bei Fehlermeldungen im Zweifel **Kernel neustarten (Kernel -> Restart)**
- "Hilfe, ich sehe den Markdown-Code" -> **Shift + Enter** in der entsprechenden Zelle
- "Hilfe, ich bekommen ResourceExhaustion / OutOfMemory (OOM) Fehler" -> **Kernel neustarten, Kernel von anderen noch laufenden Notebooks herunterfahren** (oben links auf das Jupyter-Logo klicken, dann auf den Tab "Running")
- "Hilfe, mein Notebook ist kaputt" -> siehe **Notebook "Wiederherstellung"**
- "Passiert da noch was?" -> ist der **Kreis oben rechts neben "Python 3", ausgefüllt und dunkel** dann ist der Kernel noch am Arbeiten, ist er nicht gefüllt dann ist der Kernel untätig. Die aktuell laufende Zelle ist die von oben gesehen erste bei der auf der linken Seite "In[*]" anstelle von z.B. "In[5]" steht. Es kann aber passieren dass sich der Kernel aufhängt, in dem Fall einfach oben auf __Kernel -> Interrupt__ und die Zelle erneut ausführen


### Überblick & Workflow

Die einzigen beiden Shortcuts die man sich eigentlich nur merken muss sind

- **Shift + Enter** zum Ausführen einer Zelle, und
- **Strg + Shift + p** zum Öffnen der Kommandopalette, von der aus man dann direkt Zugriff auf alle möglichen Befehle hat, inklusive entsprechender Shortcuts

Ein weiterer nützlicher Shortcut ist **Shift + o**, welcher das **Scrolling für Zellenoutput** umschaltet.

Zum **Editieren einer Zelle** genügt ein Doppelklick in die Zelle, bei Code-Zellen reicht es zum Beenden des Editiermodus einfach außerhalb der Zelle zu klicken, bei Dokumentationszellen (wie dieser hier) ist eine Ausführung der Zelle nötig um die Code-Ansicht zu verlassen.

Während eine Zelle ausgeführt wird, wechselt der Kernel-Indikator oben rechts neben "Python 3" von einem hellen Kreis mit dunklem Rand zu einem ausgefüllten dunklen Kreis und springt wieder zurück sobald die Ausführung beendet ist. Wenn mehrere Zellen gleichzeitig ausgeführt wurden, kann man an dem Label in der linken Spalte ablesen, ob die Zelle fertig ausgeführt wurde (**In [ZAHL]:**) oder ob sie gerade ausgeführt wird bzw. auf Ausführung wartet (**In [*]:**). Zusätzlich wird nach Ausführung einer Zelle die Zellenausgabe unterhalb der Zelle angezeigt.

Um den Überblick zu behalten kann es manchmal sinnvoll sein, die **Zellenausgabe zu löschen**. Dies kann u.a. auf diesen beiden Wegen erfolgen:

- oben auf Cell -> Current Outputs / All outputs -> Clear
- oben auf Kernel -> Restart & Clear Output

Der Kernel ist für die Ausführung des Python-Codes zuständig und behält den Kontext (also belegte Variablen, definierte Funktionen und belegter Speicher) seit dem letzten Kernel-(Neu)start. Dies kann zu Problemen führen wenn Zellen in anderer Reihenfolge ausgeführt werden oder Zellen übersprungen werden in denen Variablen oder Funktionen definiert werden die im weiteren Verlauf des Scripts benötigt werden, aber auch wenn **ein anderes Jupyter-Notebook gestartet wird**, da dafür ein weiterer Kernel gestartet wird.

Deswegen beim **Wechseln auf ein anderes Notebook** immer den **Kernel herunterfahren oder neustarten** (oben Kernel -> Restart/Shutdown).

# 1. Imports und Tensorflow-Setup

Neben den benötigten Imports wird wird hier Tensorflow angewiesen, nur maximal 6GB des GPU-RAMs zu belegen, damit mehrere Trainingsprozesse gleichzeitig auf jeder GPU ausgeführt werden können. Überspringt man diesen Schritt, dann reserviert Tensorflow den gesamten Grafikspeicher. Ein wichtiger Faktor für den beim Training benötigen Grafikspeicher ist die gewählte Batch-Größe, auf die später im Script noch genauer eingangen wird.

Bitte diese Zahl nicht ändern, da mehrere Trainingsprozesse auf einer GPU ausgeführt werden und es zu Crashes kommen kann wenn der GPU-Speicher voll läuft.

In [ ]:
import os
import matplotlib.pyplot as plt
import numpy as np
import PIL.Image as Image

import tensorflow as tf
print(tf.__version__)


# Tell TF to not use all GPU RAM
gpus = tf.config.experimental.list_physical_devices('GPU')
if gpus:
    try:
        tf.config.experimental.set_virtual_device_configuration(
            gpus[0],
            [tf.config.experimental.VirtualDeviceConfiguration(memory_limit=6*1024)])
        logical_gpus = tf.config.experimental.list_logical_devices('GPU')
        print(len(gpus), "Physical GPUs,", len(logical_gpus), "Logical GPUs")
    except RuntimeError as e:
    # Virtual devices must be set before GPUs have been initialized
        print(e)


from tensorflow.keras import models
from tensorflow.keras import layers
from tensorflow.keras.callbacks import Callback
from tensorflow.keras.callbacks import ModelCheckpoint

from tensorflow.keras.preprocessing.image import ImageDataGenerator

AUTOTUNE = tf.data.experimental.AUTOTUNE
import efficientnet.tfkeras as efn 

from tensorflow.python.keras import backend as K

import tensorflow_addons as tfa

print("Tensorflow setup done")

# Functions for visualization
# See https://www.tensorflow.org/tutorials/images/hub_with_keras
def image_to_network_input(filename):
    im = Image.open(filename)
    plt.imshow(np.asarray(im))

    # Convert to RGB and resize
    im_converted = im.convert(mode='RGB').resize((IMAGE_SIZE,IMAGE_SIZE))

    # Convert to array and add batch dimension
    X_test = np.array(im_converted)/255.0
    X_test = X_test[np.newaxis, ...]
    
    return X_test

def print_prediction_result(result):
    print('Predictions:')
    prediction_sum = np.sum(result)
    i = 0
    for x in np.nditer(result):
        print('{}: {:.3f}%'.format(CLASSES[i], x/prediction_sum*100))
        i+=1
        
    predicted_class = CLASSES[np.argmax(result[0], axis=-1)]
    print('Predicted class: ' + predicted_class)

def smooth(values, num_points = 100):
    xnew = np.linspace(0,len(values)-1,num_points) #100 represents number of points to make between T.min and T.max
    spl = make_interp_spline(list(range(len(values))), values, k=3) #BSpline object
    values_smooth = spl(xnew)
    
    return xnew, values_smooth
    
class LossHistory(Callback):
    def on_train_begin(self, logs={}):
        self.loss = []
        self.acc = []

    def on_batch_end(self, batch, logs={}):
        self.loss.append(logs.get('loss'))
        self.acc.append(logs.get('acc'))

def find_last(model_directory):
        """Finds the last checkpoint file of the last trained model in the
        model directory.
        Returns:
            The path of the last checkpoint file
        """
        
        checkpoints = next(os.walk(model_directory))[2]
        checkpoints = filter(lambda f: f.endswith(".hdf5"), checkpoints)
        checkpoints = sorted(checkpoints)
        if not checkpoints:
            import errno
            raise FileNotFoundError(
                errno.ENOENT, "Could not find weight files in {}".format(model_directory))
        checkpoint = os.path.join(model_directory, checkpoints[-1])
        return checkpoint

    
def show_example_predictions(dataset,IMAGES_TO_SHOW = 4, NUM_WRONG = 2):
    i = 0
    j = 0
    for image_batch, label_batch in (iter(dataset)):
        for n in range(len(image_batch)):
            if(j>=NUM_WRONG):
                if(i>=IMAGES_TO_SHOW):
                    return
            
            result=model.predict(image_batch[n][np.newaxis, ...])
            
            predicted_label=CLASSES[np.argmax(result)]
            label=class_names[label_batch[n].numpy()==1][0].title()
            
            if predicted_label.lower()!=label.lower():
                j+=1
                plt.figure(figsize=(10,10))
                ax = plt.subplot()
                plt.imshow(image_batch[n].numpy())
                plt.title("Label: "+label+"      Prediction: "+predicted_label)
                plt.axis('off')
                i+=1
            elif j>=NUM_WRONG:
                plt.figure(figsize=(10,10))
                ax = plt.subplot()
                plt.imshow(image_batch[n].numpy())
                plt.title("Label: "+label+"      Prediction: "+predicted_label)
                plt.axis('off')
                i+=1
            
            
print("Imports and setup done")

ROOT_DIR = os.path.abspath("/workspace")
DATA_DIR = os.path.abspath("/data")
MODEL_DIR = os.path.join(ROOT_DIR, "models/classification")
PRETRAINED_MODEL_DIR = os.path.abspath("/models")

# Solutions
#NUM_CLASSES =
#LEARN_RATE = 
#CONV_BASE_TRAINABLE = 

print("Directories:")
print("Root directory:", ROOT_DIR)
print("Model directory:", MODEL_DIR)
print("Pre-trained model directory:", PRETRAINED_MODEL_DIR)
print("Datasets directory:", DATA_DIR)

Die wichtigsten Ordner sind:

- `ROOT_DIR`: Das Hauptverzeichnis in dem sich die Jupyter-Notebooks befinden
- `MODEL_DIR`: Verzeichnis in dem trainierte Modelle abgespeichert werden
- `PRETRAINED_MODEL_DIR`: Verzeichnis für vortrainierte Modelle (read-only)
- `DATA_DIR`: Verzeichnis in dem sich die Datensätze befinden

Hier kann mit einfachen Linux-Kommandozeilenbefehlen die Ordnerstruktur untersucht werden

In [ ]:
!ls -l /data
!ls -l /data/MURA-v1.1
!ls -l /data/MURA-v1.1/train
!ls -l /workspace

# 2. Datensatz

Hier wird der Datensatz definiert. Es gibt in Keras und Tensorflow viele verschiedene Wege Daten einzulesen, in diesem Beispiel wird ein [tf.data.Dataset](https://www.tensorflow.org/api_docs/python/tf/data/Dataset) verwendet. Die verschiedenen Optionen zur Data Augmentation können unter dem o.g. Link nachgelesen werden und können später zur Optimierung genutzt werden.

In unserem Beispiel ist der Name des Datensatz-Ordners `MURA-v1.1` und die möglichen __Splits__ sind `train` und `valid`, die Klassen entsprechen den Ordnernamen innerhalb der Unterverzeichnisse `train/` und `valid/`.

Eine wichtige Variable hier ist `BATCH_SIZE`, welche bestimmt wie viele Bilder gleichzeitig in den Grafikspeicher geladen werden müssen. Wählt man diesen Wert zu hoch, so wird der reservierte Grafikspeicher nicht ausreichen, wählt man ihn zu niedrig so wird das Training sehr langsam. Für unser Beispiel soll dieser Wert auf 42 gesetzt werden.

In [ ]:
BATCH_SIZE = 42 # 16 with ResNet50
IMAGE_SIZE = 224 #128 with ResNet50
PRE_CROPPED_SIZE = 250 #150 with ResNet50

# Dataset directory
DATASET_DIR = '/data/MURA-v1.1/'

CLASSES = ['XR_ELBOW', 'XR_FINGER', 'XR_FOREARM', 'XR_HAND', 'XR_HUMERUS', 'XR_SHOULDER', 'XR_WRIST']
class_names=np.array(CLASSES)

#loading image paths from csv
train_paths=[]
valid_paths=[]

with open (DATASET_DIR+"train_image_paths.csv") as file:
    train_paths=file.read().splitlines()
    
with open (DATASET_DIR+"valid_image_paths.csv") as file:
    valid_paths=file.read().splitlines()  
    
train_paths=["/data/"+s for s in train_paths]
valid_paths=["/data/"+s for s in valid_paths]

print(len(train_paths),"train images")
print(len(valid_paths),"valid images")

#tensor with classnames for comparison on GPU
class_tensor=tf.constant(class_names)

#returns a label[] for given file
def get_label(file_path):
    return tf.strings.split(file_path, os.sep)[-4]==class_tensor
#prints a batch
def show_batch(image_batch, label_batch):
    plt.figure(figsize=(10,10))
    for n in range(BATCH_SIZE):
        ax = plt.subplot(5,5,n+1)
        plt.imshow(image_batch[n])
        plt.title(class_names[label_batch[n]==1][0].title())
        plt.axis('off')
        
#image to tensor
def decode_img(img):
  # convert the compressed string to a 3D uint8 tensor
    img = tf.image.decode_image(img, channels=3,expand_animations=False)
  # Use `convert_image_dtype` to convert to floats in the [0,1] range.
    img = tf.image.convert_image_dtype(img, tf.float32)
  # resize the image to the desired size.
    #return img
    return tf.image.resize(img, [PRE_CROPPED_SIZE, PRE_CROPPED_SIZE])

#loads, decodes and augments images and labels
def process_path_train(file_path):
    label = get_label(file_path)
    
  # load the raw data from the file as a string
    img = tf.io.read_file(file_path)
    img = decode_img(img)
    
    #data augmentation
    img=tf.image.random_crop(img, [IMAGE_SIZE, IMAGE_SIZE, 3],seed=42)
    img=tfa.image.rotate(img,tf.random.uniform(shape=[],dtype=tf.float32,minval=-90, maxval=90,seed=42))
    img=tf.image.random_flip_left_right(img, seed=42)
    img=tf.image.random_flip_up_down(img, seed=42)
    img=tf.image.random_brightness(img,0.4,seed=42)
    img=tf.image.random_saturation(img,0.4,2.5,seed=42)
        
    return img, label 

#no augmentation on validation set
def process_path_validation(file_path):
    label = get_label(file_path)
  # load the raw data from the file as a string
    img = tf.io.read_file(file_path)
    img = decode_img(img)
    
    img = tf.image.resize(img,[IMAGE_SIZE,IMAGE_SIZE])
        
    return img, label 

#handles image IO for GPU
def prepare_for_training(ds, cache=False, shuffle_buffer_size=1000,shuffle=True):
    if cache:
        if isinstance(cache, str):
            ds = ds.cache(cache)
        else:
            ds = ds.cache()

    if shuffle:
        ds = ds.shuffle(buffer_size=shuffle_buffer_size)
          # Repeat forever
        #ds = ds.repeat()


    ds = ds.batch(BATCH_SIZE)

  # `prefetch` lets the dataset fetch batches in the background while the model
  # is training.
    #prefetch(int)->number of images
    ds = ds.prefetch(AUTOTUNE)

    return ds


#make a dataset from files
list_train_ds = tf.data.Dataset.list_files(train_paths)
list_valid_ds = tf.data.Dataset.list_files(valid_paths)

#define operations on images
labeled_train_ds = list_train_ds.map(process_path_train,num_parallel_calls=AUTOTUNE)
labeled_valid_ds = list_valid_ds.map(process_path_validation,num_parallel_calls=AUTOTUNE)

#generate final dataset, only shuffle training data
train_ds = prepare_for_training(labeled_train_ds,shuffle=True)
valid_ds = prepare_for_training(labeled_valid_ds,shuffle=False)


print("Datensatz generiert.")

## 2.1. Bilder anzeigen lassen

Im Folgedenen werden Bilder aus dem (Validierungs-)Datensatz in Originalauflösung extrahiert. Um stattdessen Bilder aus dem Trainingsdatensatz anzuzeigen, einfach das `'valid_ds'` in der ersten Zeile zu `'train_ds'` ändern.

Die Bilder aus dem Trainingsdatensatz sind augmentiert.

In [ ]:
image_batch, label_batch = next(iter(valid_ds))
show_batch(image_batch.numpy(), label_batch.numpy())

<a id='from_scratch'></a>

# 3. Modell-Erstellung (from scratch)

Falls Sie das Training from Scratch bereits durchgeführt haben, klicken Sie [hier](#pre_trained) um zum nächsten Schritt zu springen.

Als nächstes wird das Modell erstellt. 
`conv_base` enthält das Basis-Modell, in diesem Fall ein [EfficientNet B0](https://github.com/qubvel/efficientnet), andere Möglichkeiten sind auf der [keras.io](https://keras.io/applications/)-Webseite aufgelistet (dafür müssen aber auch oben die Imports angepasst werden).

Auf das Basis-Modell wird eine neue fully-connected Schicht mit 1000 Knoten gesetzt, und darüber eine weitere fully-connected Ausgabeschicht deren Anzahl Knoten der Anzahl Klassen entspricht.

Abschließend wird das Modell kompiliert, wobei die Loss-Funktion, ein Optimizer (hier kann die Lernrate angepasst werden, s.u.), und die zu generierenden Metriken übergeben werden.

Hier wird direkt das komplette Modell trainiert, da alle Schichten zufällig initialisiert wurden und ein zweiphasiges Training daher nicht sinnvoll ist.

### Optimizer und Lernrate

Keras unterstützt [eine Reihe](https://keras.io/optimizers/) von Optimizern mit verschiedenen Parametern. Optimizer sind für die Anpassung der Gewichte zuständig. Wir werden für unser Training zwei verschiedene Optimizer nutzen:

- [Adam](https://arxiv.org/abs/1412.6980v8)
- [Stochastic Gradient Descent (SGD)](https://en.wikipedia.org/wiki/Stochastic_gradient_descent)

Die wichtigsten Parameter die von beiden Optimizern unterstützt werden:

- `lr`, die Lernrate
- `decay`, learning rate decay

Für die Lernrate liegen übliche zwischen `0.001` und `0.00001`. Größere Lernrate ermöglichen schnelleres Training, können aber auch zu problematischen Sprüngen im Lösungsraum und Divergenz (Fehler gleichbleibend schlecht) führen. Kleinere Lernraten eignen sich für das Fine-Tuning.

Der learning rate decay sorgt dafür, dass die Lernrate im Laufe des Trainings immer geringer wird. Die Idee ist mit einer großen Lernrate zu starten um so schnell in die Nähe eines lokalen Minimums zu gelangen, und dann mit einer geringeren Lernrate diesem lokalen Minimum möglichst nahe zu kommen. Der Wert sollte hier abhängig von der Anzahl Epochen und Batches gewählt werden. Die Formel für die Berechnung der aktuellen Lernrate ist

\begin{equation*}
l = l_{init} * \frac{1}{1 + d * i}
\end{equation*}

mit $l$ = Lernrate, $l_{init}$ = initiale Lernrate, $d$ = decay, $i$ = Iterationen (Anzahl verarbeiteter Batches).

Hier ein paar Beispiele mit verschiedenen Lernraten und $d = 0.00025$:

<style type="text/css" rel="stylesheet">
.foo table { width: 300px; }
</style>
<div class="foo">
    
| Epoche | &nbsp; &nbsp; &nbsp; &nbsp; &nbsp; &nbsp; &nbsp; &nbsp; Formel &nbsp; &nbsp; &nbsp; &nbsp; &nbsp; &nbsp; &nbsp; &nbsp; | $l$ bei $l_{init} = 0.001$ | $l$ bei $l_{init} = 0.0005$ |
| ---------- | ---------- | ---------- | ---------- |
| 0 | $l_{init} * \frac{1}{1 + 0.00025 * 0*2300}$ | &nbsp; &nbsp; 0.001 &nbsp; &nbsp; &nbsp; &nbsp; &nbsp; &nbsp; | &nbsp; &nbsp; &nbsp; 0.0005 &nbsp; &nbsp; &nbsp; &nbsp; &nbsp; |
| 1 | $l_{init} * \frac{1}{1 + 0.00025 * 1*2300}$ | &nbsp; &nbsp; &nbsp; &nbsp; &nbsp; &nbsp; &nbsp; &nbsp; 0.000634921 | &nbsp; &nbsp; &nbsp; &nbsp; &nbsp; &nbsp; &nbsp; 0.00031746 &nbsp; |
| 2 | $l_{init} * \frac{1}{1 + 0.00025 * 2*2300}$ | &nbsp; &nbsp; &nbsp; &nbsp; &nbsp; &nbsp; &nbsp; &nbsp; 0.000465116 | &nbsp; &nbsp; &nbsp; &nbsp; &nbsp; &nbsp; &nbsp; &nbsp;  0.000232558 |
| 3 | $l_{init} * \frac{1}{1 + 0.00025 * 3*2300}$ | &nbsp; &nbsp; &nbsp; &nbsp; &nbsp; &nbsp; &nbsp; &nbsp; 0.000366972 | &nbsp; &nbsp; &nbsp; &nbsp; &nbsp; &nbsp; &nbsp; &nbsp;  0.000183486 |
| 4 | $l_{init} * \frac{1}{1 + 0.00025 * 4*2300}$ | &nbsp; &nbsp; &nbsp; &nbsp; &nbsp; &nbsp; &nbsp; &nbsp; 0.000303030 | &nbsp; &nbsp; &nbsp; &nbsp; &nbsp; &nbsp; &nbsp; &nbsp;  0.000151515 |
| 5 | $l_{init} * \frac{1}{1 + 0.00025 * 5*2300}$ | &nbsp; &nbsp; &nbsp; &nbsp; &nbsp; &nbsp; &nbsp; &nbsp; 0.000258065 | &nbsp; &nbsp; &nbsp; &nbsp; &nbsp; &nbsp; &nbsp; &nbsp;  0.000129032 |

</div>

Aufgaben:

- Ersetzen Sie die Variable `NUM_CLASSES` durch die Anzahl Klassen
- Ersetzen Sie die Variable `LEARN_RATE` durch eine geeignete Lernrate

In [ ]:
##
# Create model
##

print("Creating model")

os.makedirs(MODEL_DIR, 0o777, exist_ok=True)
print("model_dir: ", os.path.realpath(MODEL_DIR))

model_file = os.path.join(MODEL_DIR, "model_from_scratch.h5")

# Create Keras model
# see https://keras.io/applications/
conv_base = efn.EfficientNetB0(
                weights=None,
                include_top=False,
                input_shape=(IMAGE_SIZE, IMAGE_SIZE, 3)
            )

model = models.Sequential()
model.add(conv_base)
model.add(layers.Flatten())
# Add more layers here if needed
model.add(layers.Dense(1000, activation='relu'))
# The number of nodes in the last layer should be equal to the number of classes
model.add(layers.Dense(NUM_CLASSES, activation='softmax'))
# Enable training for the base model
conv_base.trainable = True

# Here you can adjust the learning rate
# see https://keras.io/optimizers/
optimizer = tf.keras.optimizers.Adam(lr=LEARN_RATE, decay=0.0025)

model.compile(loss='categorical_crossentropy',
              optimizer=optimizer,
              metrics=['accuracy'])

print("Model successfully created")


## 3.1 Training (from scratch)

Das eigentliche Training wird über die Funktion `fit` gestartet, welchem die zuvor erstellten Datensätze übergeben werden.

Dies kann einige Zeit dauern. Ob der Kernel im Hintergrund noch aktiv ist lässt sich in dem Kreis oben rechts erkennen – solange der Kreis ausgefüllt ist, läuft das Script noch.

In [ ]:
print("Start training from scratch")
#filepath = os.path.join(MODEL_DIR, "from_scratch_{epoch:02d}-{val_loss:.2f}.hdf5")
#model_checkpoint = ModelCheckpoint(filepath)
history=model.fit(
        train_ds, 
        epochs=2,
        validation_data=valid_ds
        )

#models.save_model(model, filepath = model_file)

#print("Finished training, model saved to: " + MODEL_DIR)

Die Ausgabe des Trainingsscripts enthält folgende Informationen:

- `8/2300`: Das aktuelle Minibatch. Wie oben beschrieben ergibt sich die Zahl 30 aus der Anzahl Trainingsbilder durch die Batch-Größe: `36800 / 16 = 2300`
- `ETA: 1:55`: Erwartete Restzeit
- `413s 180ms/step`: Gesamtdauer für die Trainingsepoche sowie durchschnittliche Dauer eines Minibatches
- `loss: 1.3854`: Wert der zu minimierenden Verlustfunction (in diesem Fall [Cross Entropy](https://en.wikipedia.org/wiki/Cross_entropy)) auf dem letzten Minibatch
- `acc: 0.4926`: Accuracy auf dem aktuellen Trainings-Minibatch, also prozentualer Anteil korrekt klassifizierter Bilder
- `val_loss: 1.5387`: Wert der Verlustfunction auf den gesamten Validierungsdaten, wird nur einmal pro Epoche generiert
- `val_acc: 0.5148`: Accuracy auf den gesamten Validierungsdaten, wird nur einmal pro Epoche generiert

Wichtig ist hier ein Vergleich zwischen den Werten für die Trainings- und Validierungsdaten. Wenn diese Werte weit auseinander gehen dann liegt Overfitting vor, die Trainingsdaten werden also auswendig gelernt.

<a id='visualization'></a>
## 3.2 Visualisierung (from scratch)

Nach dem Training kann mit der `LossHistory`-Instanz eine Visualisierung der Klassifikationsergebnisse stattfinden.

Es werden hier nur die Loss und Accuracy der Trainingsbatches angezeigt, da der Validierungsfehler nur einmal am Ende jeder Epoche (in diesem Fall also insgesmat nur ein mal) berechnet wird.

In [ ]:
#insert values if history contains only 1 epoch to plot graph
if len(history.history['accuracy'])==1:
    for i in history.history:
        history.history[i].insert(0,history.history[i][0])

# summarize history for accuracy
plt.plot(history.history['accuracy'])
plt.plot(history.history['val_accuracy'])
plt.title('model accuracy')
plt.ylabel('accuracy')
plt.xlabel('epoch')
plt.legend(['train', 'test'], loc='upper left')
plt.show()
# summarize history for loss
plt.plot(history.history['loss'])
plt.plot(history.history['val_loss'])
plt.title('model loss')
plt.ylabel('loss')
plt.xlabel('epoch')
plt.legend(['train', 'test'], loc='upper left')
plt.show()

### 3.2.1 Einzelinferenz (Trainingsbilder, from scratch)

Spannender hingegen ist die anschließende Einzelinferenz: was genau macht unser Netz aus bestimmten Bildern der Trainingsmenge?

Es werden hier vier (Variable `IMAGES_TO_SHOW`) zufällige Bilder aus der Trainingsmenge genommen.

Über die Variable `NUM_WRONG` kann festgelegt werden, wie viele falsch klassifizierte Bilder mindestens ausgewählt werden sollen.

In [ ]:
show_example_predictions(train_ds,IMAGES_TO_SHOW = 4, NUM_WRONG = 2)

Hier wird das gleiche für Bilder aus dem Validierungsdatensatz durchgeführt.

In [ ]:
show_example_predictions(valid_ds,IMAGES_TO_SHOW = 4, NUM_WRONG = 2)

<a id='pre_trained'></a>
# 4. Modell-Erstellung (pre-trained)

Nachdem wir uns das Training from scratch angesehen haben soll nun ein Training basierend auf einem vortrainierten Modell stattfinden. Die in Keras mitgelieferten Modelle sind mit dem ImageNet-Datensatz vortrainiert. Um ein vortrainiertes Netz zu nutzen muss bei der Erstellung der `conv_base` der `weights` Parameter auf `imagenet` gesetzt werden.

Wichtig ist hierbei `conv_base.trainable` zunächst auf `False` zu setzen, wodurch nur die neu hinzugefügten Schichten für eine Epoche trainierten werden.

Falls der Validierungsfehler nach dem ersten Trainingsschritt sehr schlecht ist, dann ist das kein Grund zur Sorge – dieser wird spätestens nach dem zweiten Trainingsschritt dem Trainingsfehler sehr ähnlich sein.

Falls ein `ResourceExhaustion` bzw. `OOM`-Error auftritt, muss der Kernel neustartet werden `"restart the kernel (with dialog)"`. Anschließend alle Zellen einschließlich der Datensatzerstellung (aber nicht das Training from scratch) ausführen und dann hier fortfahren. Klicken Sie dazu [hier](#from_scratch), dann oben auf `Cell -> Run all above`, und anschließend auf den Link zum Überspringen des Schrittes, um wieder hierhin zu gelangen.

In [ ]:
print('Creating pre-trained model, this may take a while...')

# Clear session memory to avoid OOM
K.clear_session()

# Create Keras model
# see https://keras.io/applications/
conv_base = efn.EfficientNetB0(weights='noisy-student',
                  include_top=False,
                  input_shape=(IMAGE_SIZE, IMAGE_SIZE, 3))

model = models.Sequential()
model.add(conv_base)
model.add(layers.Flatten())
# Add more layers here if needed
model.add(layers.Dense(1000, activation='relu'))
# The number of nodes in the last layer should be equal to the number of classes
model.add(layers.Dense(7, activation='softmax'))

print("Model created")

## 4.1 Training neue Schichten (pre-trained)


Aufgaben:

- Frieren Sie die Gewichte der vortrainierten Schichten über den Parameter `conv_base.trainable` ein, indem Sie die Variable `CONV_BASE_TRAINABLE` durch `True` oder `False` ersetzen

In [ ]:
# Disable training for the pre-trained model
conv_base.trainable = False

# Here you can adjust the learning rate
# see https://keras.io/optimizers/
optimizer = tf.keras.optimizers.Adam(lr=0.0001, decay=0.00025, clipnorm=1.)

model.compile(loss='categorical_crossentropy',
              optimizer=optimizer,
              metrics=['accuracy'])

print('Training new model')


print("Start training: only new layers")
# First train only top layers
history=model.fit(
        train_ds, 
        epochs=1,
        validation_data=valid_ds
        )

#models.save_model(model, filepath = model_file)
#evaluation = model.evaluate_generator(train_generator, NUM_TRAINING_SAMPLES // BATCH_SIZE)
#print("Training loss:", evaluation[0], "\nTraining accuracy: {:.2f}%".format(evaluation[1]*100))
#evaluation = model.evaluate_generator(validation_generator, NUM_VALIDATION_SAMPLES // BATCH_SIZE)
#print("Validation loss:", evaluation[0], "\nValidation accuracy: {:.2f}%".format(evaluation[1]*100))
#print("Finished training, model saved to: " + MODEL_DIR)

<a id='visualization'></a>
## 4.2 Visualisierung nach Training neuer Schichten (pre-trained)

Nach dem Training kann mit der `LossHistory`-Instanz eine Visualisierung der Klassifikationsergebnisse stattfinden.

Diese Visualisierung zeigt jederzeit den Verlauf des zuletzt durchgeführten Trainings an, Sie können also nach jedem Training hierhin zurück kehren und diesen Code-Block erneut ausführen.

In [ ]:
#insert values if history contains only 1 epoch to plot graph
if len(history.history['accuracy'])==1:
    for i in history.history:
        history.history[i].insert(0,history.history[i][0])

# summarize history for accuracy
plt.plot(history.history['accuracy'])
plt.plot(history.history['val_accuracy'])
plt.title('model accuracy')
plt.ylabel('accuracy')
plt.xlabel('epoch')
plt.legend(['train', 'test'], loc='upper left')
plt.show()
# summarize history for loss
plt.plot(history.history['loss'])
plt.plot(history.history['val_loss'])
plt.title('model loss')
plt.ylabel('loss')
plt.xlabel('epoch')
plt.legend(['train', 'test'], loc='upper left')
plt.show()

### 4.2.1 Einzelinferenz nach Training neuer Schichten (pre-trained)

Spannender hingegen ist die anschließende Einzelinferenz: was genau macht unser Netz aus bestimmten Bildern der Trainingsmenge?

Es werden hier vier (Variable `IMAGES_TO_SHOW`) zufällige Bilder aus der Trainingsmenge genommen.

Über die Variable `NUM_WRONG` kann festgelegt werden, wie viele falsch klassifizierte Bilder mindestens ausgewählt werden sollen.

In [ ]:
show_example_predictions(train_ds,IMAGES_TO_SHOW = 4, NUM_WRONG = 2)

Hier wird das gleiche für Bilder aus dem Validierungsdatensatz durchgeführt.

In [ ]:
show_example_predictions(valid_ds,IMAGES_TO_SHOW = 5, NUM_WRONG = 2)

## 4.3 Fine-Tuning (pre-trained)


Im zweiten Schritt wird nun das gesamte Modell für eine Epoche trainiert (`conv_base.trainable` wird auf `True` gesetzt).

In [ ]:
conv_base.trainable = True

optimizer = tf.keras.optimizers.Adam(lr=0.00001, decay=0.00025, clipnorm=1.)

model.compile(loss='categorical_crossentropy',
              optimizer=optimizer,
              metrics=['accuracy'])

try: history
except NameError: history = LossHistory()


print("Start training from pre-trained: fine-tuning all layers")
history=model.fit(
        train_ds, 
        epochs=20,
        validation_data=valid_ds
        )

#evaluation = model.evaluate_generator(train_generator, NUM_TRAINING_SAMPLES // BATCH_SIZE)
#print("Training loss:", evaluation[0], "\nTraining accuracy: {:.2f}%".format(evaluation[1]*100))
#evaluation = model.evaluate_generator(validation_generator, NUM_VALIDATION_SAMPLES // BATCH_SIZE)
#print("Validation loss:", evaluation[0], "\nValidation accuracy: {:.2f}%".format(evaluation[1]*100))
#print("Finished training, model saved to: " + MODEL_DIR)

## 4.4 Visualisierung Fine-Tuning (pre-trained)

In [ ]:
#insert values if history contains only 1 epoch to plot graph
if len(history.history['accuracy'])==1:
    for i in history.history:
        history.history[i].insert(0,history.history[i][0])

# summarize history for accuracy
plt.plot(history.history['accuracy'])
plt.plot(history.history['val_accuracy'])
plt.title('model accuracy')
plt.ylabel('accuracy')
plt.xlabel('epoch')
plt.legend(['train', 'test'], loc='upper left')
plt.show()
# summarize history for loss
plt.plot(history.history['loss'])
plt.plot(history.history['val_loss'])
plt.title('model loss')
plt.ylabel('loss')
plt.xlabel('epoch')
plt.legend(['train', 'test'], loc='upper left')
plt.show()

Führen Sie nun die finale Einzelinferenz auf den Trainings-/Validierungsdaten selbst durch

## Out of Class Bilder

Zum Test wie unser Modell mit Bildern umgeht die es noch nie gesehen hat, geben wir ihm nun das folgende Schädel-CT-Bild:

<img src="https://prod-images.static.radiopaedia.org/images/17058746/a60ee77e6b44da2d72a25cea08dfd2_big_gallery.jpeg" />
Quelle: radiopaedia.org

In [ ]:
example_generator = ImageDataGenerator().flow_from_directory(DATA_DIR + '/ooc', batch_size=1, target_size=(IMAGE_SIZE,IMAGE_SIZE), class_mode='categorical')
i=0
for x_batch, y_batch in example_generator:
    
    x_input = x_batch[0]/255.
    x_input = x_input[np.newaxis, ...]
    result = model.predict(x_input)
    
    predicted_class = CLASSES[np.argmax(result[0], axis=-1)]
    correct_class = CLASSES[np.where(y_batch[0] == 1.)[0][0]]
    
    print("\n\nImage", (i+1))
    plt.figure()
    plt.imshow(x_batch[0].astype(int))
    
    
    print_prediction_result(result)
    plt.show()
    i+=1
    break

# Results
## Run 1
Eine Epoche neue Schichten trainiert:<br/>
optimizer = tf.keras.optimizers.Adam(lr=0.001, decay=0.0025, clipnorm=1.)<br/>
``loss: 1.9823 - accuracy: 0.7639 - val_loss: 0.6412 - val_accuracy: 0.7879``

10 Epochen das gesamte Netzwerk trainiert:<br/>
optimizer = tf.keras.optimizers.SGD(lr=0.0001, decay=0.00025, clipnorm=1.)
``loss: 0.2861 - accuracy: 0.9077 - val_loss: 0.3449 - val_accuracy: 0.8943``

## Run 2
Es wurde das Efficientnet B4 mit einer Bildgröße von 380x380 verwendet. Anschließend wurden 20 Epochen trainiert.<br/>
Pretraining:<br/>
``loss: 0.8445 - accuracy: 0.7323 - val_loss: 0.7250 - val_accuracy: 0.7482``

Finetuning:<br/>
``loss: 0.1384 - accuracy: 0.9584 - val_loss: 0.1925 - val_accuracy: 0.9537``<br/>
Das Training dauerte nur zwei Stunden.